In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import catboost as cb
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, StratifiedKFold,TimeSeriesSplit,KFold,GroupKFold
import xgboost as xgb
from datetime import datetime
from kaggle.competitions import nflrush
import math
import tqdm
from scipy.special import expit
from scipy.spatial import Delaunay, delaunay_plot_2d, Voronoi, voronoi_plot_2d, ConvexHull
pd.set_option("display.max_rows",1000)
env = nflrush.make_env()

In [2]:
def crps_score(y_pred, data):
    y_pred = y_pred.reshape(-1,199)
    y_true = np.array(data.get_label()).astype("int")
    y_true_cdf = np.zeros([y_true.shape[0], 199])
    for i, y in enumerate(y_true):
        y_true_cdf[i, y] = 1
    y_true_cdf = np.clip(np.cumsum(y_true_cdf, axis=1), 0, 1)
    y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
    return "CRPS", ((y_true_cdf - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * y_true.shape[0]), False

def simple_crps(y_true, y_pred):
    y_true_cdf = np.zeros([y_true.shape[0], 199])
    for i, y in enumerate(y_true):
        y_true_cdf[i, y] = 1
    y_true_cdf = np.clip(np.cumsum(y_true_cdf, axis=1), 0, 1)
    y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
    return ((y_true_cdf - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * y_true.shape[0])

In [3]:
train = pd.read_csv('../input/nfl-big-data-bowl-2020/train.csv',low_memory=False)

# preprocess and feature engineering 

In [4]:
def transform_time_quarter(str1):
    return int(str1[:2])*60 + int(str1[3:5])
  
def transform_time_all(str1,quarter):
    if quarter<=4:
        return 15*60 - (int(str1[:2])*60 + int(str1[3:5])) + (quarter-1)*15*60
    if quarter ==5:
        return 10*60 - (int(str1[:2])*60 + int(str1[3:5])) + (quarter-1)*15*60
      
def back_direction(orientation):
    if orientation > 180.0:
        return 1
    else:
        return 0
    
def transform_height(te):
    return (int(te.split('-')[0])*12 + int(te.split('-')[1]))*2.54/100

def voronoi_volumes(points, selected_index):
    v = Voronoi(points)
    vol = np.zeros(v.npoints)
      
    for i, reg_num in enumerate(v.point_region):
        if reg_num == v.point_region[selected_index]:
            indices = v.regions[reg_num]
            if -1 in indices: # some regions can be opened
                vol = -999 ## insert missing value when the area is open
            else:
                vol = ConvexHull(v.vertices[indices]).volume      
            break
    return vol

def radius_calc(dist_to_ball):
    return 4 + 6 * (dist_to_ball >= 15) + (dist_to_ball ** 3) / 560 * (dist_to_ball < 15)

def compute_influence_to_ball(row):
    point = np.array([row["RushX"], row["RushY"]])
    theta = math.radians(row['Orientation'])
    speed = row['S']
    player_coords = row[['X', 'Y']].values
    dist_to_ball = row["DisToRusher"]    

    S_ratio = (speed / 13) ** 2    # we set max_speed to 13 m/s ((dominance_df["S"]*0.9144).max() #8.64m/s)
    RADIUS = radius_calc(dist_to_ball)  # updated

    S_matrix = np.matrix([[RADIUS * (1 + S_ratio), 0], [0, RADIUS * (1 - S_ratio)]])
    R_matrix = np.matrix([[np.cos(theta), - np.sin(theta)], [np.sin(theta), np.cos(theta)]])
    COV_matrix = np.dot(np.dot(np.dot(R_matrix, S_matrix), S_matrix), np.linalg.inv(R_matrix))
    
    norm_fact = (1 / 2 * np.pi) * (1 / np.sqrt(np.linalg.det(COV_matrix)))    
    mu_play = player_coords + speed * np.array([np.cos(theta), np.sin(theta)]) / 2
    
    intermed_scalar_player = np.dot(np.dot((player_coords - mu_play),
                                    np.linalg.inv(COV_matrix)),
                             np.transpose((player_coords - mu_play)))
    player_influence = norm_fact * np.exp(- 0.5 * intermed_scalar_player[0, 0])
    
    intermed_scalar_point = np.dot(np.dot((point - mu_play), 
                                    np.linalg.inv(COV_matrix)), 
                             np.transpose((point - mu_play)))
    point_influence = norm_fact * np.exp(- 0.5 * intermed_scalar_point[0, 0])
    
    return point_influence / player_influence

In [5]:
remove_features = ['GameId','PlayId','DisplayName','GameClock','TimeHandoff','TimeSnap', 'PlayDirection', 'TeamOnOffense', 
                    'PlayerBirthDate', 'is_run', 'NflIdRusher', 'date_game', 'RushX', 'RushY', 'PossessionTeam', 
                   'FieldPosition', 'Position', 'PlayerHeight', 'HomeTeamAbbr', 'VisitorTeamAbbr', 'Turf', 'Quarter']
top20_weather = list(train.GameWeather.value_counts(normalize=True, dropna=False).cumsum().head(20).index)

In [6]:
def transform_data(df):
    df.loc[df.VisitorTeamAbbr == "ARI",'VisitorTeamAbbr'] = "ARZ"
    df.loc[df.HomeTeamAbbr == "ARI",'HomeTeamAbbr'] = "ARZ"

    df.loc[df.VisitorTeamAbbr == "BAL",'VisitorTeamAbbr'] = "BLT"
    df.loc[df.HomeTeamAbbr == "BAL",'HomeTeamAbbr'] = "BLT"

    df.loc[df.VisitorTeamAbbr == "CLE",'VisitorTeamAbbr'] = "CLV"
    df.loc[df.HomeTeamAbbr == "CLE",'HomeTeamAbbr'] = "CLV"

    df.loc[df.VisitorTeamAbbr == "HOU",'VisitorTeamAbbr'] = "HST"
    df.loc[df.HomeTeamAbbr == "HOU",'HomeTeamAbbr'] = "HST"

    df['is_run'] = df.NflId == df.NflIdRusher

    df['S'] = 10 * df['Dis']
    df.loc[df.Season == 2017, 'A'] = df[df.Season == 2018]['A'].mean()
    #df['A'] = (df['Dis'] / df['S']) / 0.1
    
    df['ToLeft'] = df.PlayDirection == "left"
    df['TeamOnOffense'] = "home"
    df.loc[df.PossessionTeam != df.HomeTeamAbbr, 'TeamOnOffense'] = "away"
    df['OnOffense'] = df.Team == df.TeamOnOffense 
    df['YardLine_std'] = 100 - df.YardLine.copy()
    df.loc[df.FieldPosition.fillna('') == df.PossessionTeam,  
            'YardLine_std'
             ] = df.loc[df.FieldPosition.fillna('') == df.PossessionTeam,  
              'YardLine']
    df['X_std'] = df.X.copy()
    df.loc[df.ToLeft, 'X_std'] = 120 - df.loc[df.ToLeft, 'X'] 
    df['Y_std'] = df.Y.copy()
    df.loc[df.ToLeft, 'Y_std'] = 53.3 - df.loc[df.ToLeft, 'Y'] 
    df['Orientation_std'] = df.Orientation.copy()
    df.loc[df.ToLeft, 'Orientation_std'] = np.mod(180 + df.loc[df.ToLeft, 'Orientation_std'], 360)
    df['Dir_std'] = df.Dir.copy()
    df.loc[df.ToLeft, 'Dir_std'] = np.mod(180 + df.loc[df.ToLeft, 'Dir_std'], 360)
    df.loc[df['Season'] == 2017, 'Orientation_std'] = np.mod(90 + df.loc[df['Season'] == 2017, 'Orientation_std'], 360) 
    df.drop(["X", "Y", "Orientation", "YardLine", "Dir", "ToLeft"], axis=1, inplace=True)
    df.rename(columns={'X_std': 'X', 'Y_std': 'Y', 'Orientation_std': 'Orientation', 'Dir_std': 'Dir', "YardLine_std": "YardLine"}, inplace=True)
    
    df['date_game'] = df.GameId.map(lambda x:pd.to_datetime(str(x)[:8]))
    df['age'] = (df.date_game.map(pd.to_datetime) - df.PlayerBirthDate.map(pd.to_datetime)).map(lambda x:x.days)/365

    df["Momentum"] = df["S"] * df["PlayerWeight"]

    rusher_x = np.array(df.groupby(["PlayId", "is_run"])["X"].agg(np.mean)[1::2])
    df["RushX"] = np.repeat(rusher_x, 22) # repeat each elemnt 22 times
    rusher_y = np.array(df.groupby(["PlayId", "is_run"])["Y"].agg(np.mean)[1::2])
    df["RushY"] = np.repeat(rusher_y, 22) 
    df["DisToRusher"] = np.sqrt((df["X"] - df["RushX"]) ** 2 + (df["Y"] - df["RushY"]) ** 2)
    df["TackleTimeToRusher"] = df["DisToRusher"] / df["S"] # includes nan when the speed of rusher is 0
    #df.loc[df.is_run==True, "TackleTimeToRusher"] = 0
    
    df["Dir_cos"] = df["Dir"].apply(lambda x : np.cos((450-x) * np.pi/ 180))
    
    df["Momentum_cos"] = df["Momentum"] * df["Dir_cos"]
    
    df["AttackAngle"] =  np.arctan((df["Y"] - df["RushY"]) / (df["X"] - df["RushX"])) * 180 / np.pi
    
    df_single = df[df.is_run==True].copy()
        
    df_single['time_end'] = df_single.apply(lambda x:transform_time_all(x.loc['GameClock'],x.loc['Quarter']),axis=1)

    df_single["Stadium"] = df_single["Stadium"].map(lambda x: "Broncos Stadium at Mile High" if x=="Broncos Stadium At Mile High" 
                                             else ("CenturyLink Field" if x == "CenturyField" or x == x=="CenturyLink"
                                             else ("Everbank Field" if x == "EverBank Field"
                                             else ("FirstEnergy Stadium" if x =="First Energy Stadium" or x=="FirstEnergy" or x == "FirstEnergyStadium"
                                             else ("Lambeau Field" if x == "Lambeau field"
                                             else ("Los Angeles Memorial Coliseum" if x == "Los Angeles Memorial Coliesum"
                                             else ("M&T Bank Stadium" if x == "M & T Bank Stadium" or x == "M&T Stadium"
                                             else ("Mercedes-Benz Superdome" if x == "Mercedes-Benz Dome"
                                             else ("MetLife Stadium" if x == "MetLife" or x == "Metlife Stadium"
                                             else ("NRG Stadium" if x == "NRG"
                                             else ("Oakland-Alameda County Coliseum" if x == "Oakland Alameda-County Coliseum"
                                             else ("Paul Brown Stadium" if x == "Paul Brown Stdium"
                                             else ("Twickenham Stadium" if x == "Twickenham" else x)))))))))))))

    df_single["Location"] = df_single["Location"].map(lambda x: "Arlington, TX" if x == "Arlington, Texas"
                                            else ("Baltimore, MD" if x == "Baltimore, Maryland" or x == "Baltimore, Md."
                                            else ("Charlotte, NC" if x == "Charlotte, North Carolina"
                                            else ("Chicago, IL" if x == "Chicago. IL"
                                            else ("Cincinnati, OH" if x == "Cincinnati, Ohio"
                                            else ("Cleveland, OH" if x == "Cleveland" or x == "Cleveland Ohio" or x == "Cleveland, Ohio" or x == "Cleveland,Ohio"
                                            else ("Detroit, MI" if x == "Detroit"
                                            else ("East Rutherford, NJ" if x == "E. Rutherford, NJ" or x == "East Rutherford, N.J."
                                            else ("Foxborough, MA" if x == "Foxborough, Ma"
                                            else ("Houston, TX" if x == "Houston, Texas"
                                            else ("Jacksonville, FL" if x == "Jacksonville Florida" or x == "Jacksonville, Fl" or x == "Jacksonville, Florida"
                                            else ("London" if x == "London, England"
                                            else ("Los Angeles, CA" if x == "Los Angeles, Calif."
                                            else ("Miami Gardens, FLA" if x == "Miami Gardens, Fla."
                                            else ("New Orleans, LA" if x == "New Orleans" or x == "New Orleans, La."
                                            else ("Orchard Park, NY" if x == "Orchard Park NY"
                                            else ("Philadelphia, PA" if x == "Philadelphia, Pa."
                                            else ("Pittsburgh, PA" if x == "Pittsburgh"
                                            else ("Seattle, WA" if x == "Seattle" else x)))))))))))))))))))

    grass_labels = ['grass', 'natural grass', 'natural', 'naturall grass']
    df_single['Grass'] = np.where(df_single.Turf.str.lower().isin(grass_labels), "Natural", "Artificial")
                                                                                                                         
    df_single["OffenseFormation"] = df_single["OffenseFormation"].fillna("Unknown") 
    df_single['DefendersInTheBox_vs_Distance'] = df_single['DefendersInTheBox'] / df_single['Distance']
                                                                   
    df_single["DisToQB"] = np.array(df[(df.Position=="QB") | (df.Position=="C")].groupby(["PlayId"]).agg(np.mean)["DisToRusher"])

    df_single['runner_height'] = df_single.PlayerHeight.map(transform_height)
    df_single.drop(remove_features,axis=1,inplace=True) 

    tmp = df.groupby(["PlayId", "OnOffense"]).agg(np.mean)[["X", "Y", "Momentum", "Momentum_cos"]]
    df_single["DefenseAveX"] = np.array(tmp[0::2]["X"])
    df_single["OffenseAveX"] = np.array(tmp[1::2]["X"])
    df_single["DefenseAveY"] = np.array(tmp[0::2]["Y"]) 
    df_single["OffenseAveY"] = np.array(tmp[1::2]["Y"]) 
    df_single["DefenseAveMomentum"] = np.array(tmp[0::2]["Momentum"])
    df_single["OffenseAveMomentum"] = np.array(tmp[1::2]["Momentum"])
    df_single["DefenseAveMomentum_cos"] = np.array(tmp[0::2]["Momentum_cos"])
    df_single["OffenseAveMomentum_cos"] = np.array(tmp[1::2]["Momentum_cos"])
    
    tmp = df.groupby(["PlayId", "OnOffense"]).agg(["std"])[["X", "Y", "Momentum", "Momentum_cos"]]
    df_single["DefenseStdX"] = np.array(tmp[0::2]["X"])
    df_single["OffenseStdX"] = np.array(tmp[1::2]["X"])
    df_single["DefenseStdY"] = np.array(tmp[0::2]["Y"])
    df_single["OffenseStdY"] = np.array(tmp[1::2]["Y"])
    df_single["DefenseStdMomentum"] = np.array(tmp[0::2]["Momentum"])
    df_single["OffenseStdMomentum"] = np.array(tmp[1::2]["Momentum"])
    df_single["DefenseStdMomentum_cos"] = np.array(tmp[0::2]["Momentum_cos"])
    df_single["OffenseStdMomentum_cos"] = np.array(tmp[1::2]["Momentum_cos"])
    
    df_single["RunnerToDefenseCentoid"] = np.sqrt((df_single["X"] - df_single["DefenseAveX"]) ** 2 + (df_single["Y"] - df_single["DefenseAveY"]) ** 2)
    df_single["RunnerToOffenseCentoid"] = np.sqrt((df_single["X"] - df_single["OffenseAveX"]) ** 2 + (df_single["Y"] - df_single["OffenseAveY"]) ** 2)

    tmp_max = df.groupby(["PlayId", "OnOffense"])["X"].max()
    tmp_min = df.groupby(["PlayId", "OnOffense"])["X"].min()
    df_single["DefenseSpreadX"] = np.array(tmp_max[0::2]- tmp_min[0::2])
    df_single["OffenseSpreadX"] = np.array(tmp_max[1::2]- tmp_min[1::2])

    df_single["RunnerToScrimmage"] = df_single["X"] - (df_single["YardLine"] + 10)

    df_single["MinTackleTime"] = np.array(df.groupby(["PlayId", "OnOffense"])["TackleTimeToRusher"].min()[0::2])
    df_single["1stDefender_Momentum_cos"] = np.array(df.loc[df.groupby(["PlayId", "OnOffense"])["DisToRusher"].idxmin()[0::2]]["Momentum_cos"])
    first_offenser_index =df.groupby(["PlayId", "OnOffense"])["DisToRusher"].nsmallest(2)[3::4].reset_index()["level_2"]
    df_single["1stOffenser_Momentum_cos"] = np.array(df.loc[first_offenser_index]["Momentum_cos"])
    df_single["1stDefender_AttackAngle"] = np.array(df.loc[df.groupby(["PlayId", "OnOffense"])["DisToRusher"].idxmin()[0::2]]["AttackAngle"])

    pts = np.array(df[["X", "Y"]]).reshape(df.shape[0]//22, 22, 2) # plays * players * (X, Y, rusher)
    rusher_index = list(df[df.is_run==True].index % 22) 
    closest_def_index = list(df.loc[df.groupby(["PlayId", "OnOffense"])["DisToRusher"].idxmin()[0::2]].index % 22)
    rusher_voronoi = []
    closest_def_voronoi = []

    for i in range(0, df.shape[0] //22):
        rusher_voronoi.append(voronoi_volumes(pts[i], rusher_index[i]))
        closest_def_voronoi.append(voronoi_volumes(pts[i], closest_def_index[i]))
    df_single["RusherVoronoi"] = rusher_voronoi    
    df_single["FirstDefenderVoronoi"] = closest_def_voronoi 
        
    df_single.fillna(-999,inplace=True) 
    remove_features2 = ["OnOffense", "DisToRusher", "TackleTimeToRusher", "OffenseAveX", "OffenseAveY"]#"InfluToBall",, "RatioSToRusher"]
    df_single.drop(remove_features2, axis=1, inplace=True)

    return df_single

In [7]:
%%time
train_single = transform_data(train)
y_train = train_single.Yards + 99 # to categorize
X_train = train_single.drop(['Yards'],axis=1)
for f in X_train.columns:
    if X_train[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f])+[-999])
        X_train[f] = lbl.transform(list(X_train[f]))

/opt/conda/lib/python3.6/site-packages/pandas/core/series.py:856: RuntimeWarning: invalid value encountered in arctan
  result = getattr(ufunc, method)(*inputs, **kwargs)


CPU times: user 5min 26s, sys: 2.66 s, total: 5min 29s
Wall time: 5min 27s


# modelling

In [8]:
# my previous version
#n_folds=5
#kf=KFold(n_splits = n_folds, random_state=1125)
#average_crps = 0
#models = []
#crps_scores = []
#xgb_params = {
#    "objective" : "multi:softprob",
#    "eval_metric" : "mlogloss", 
#    "max_depth" : 4,
#    "boosting": 'gbdt',
#    "num_class": 199,
#    "num_leaves" : 13,
#    "learning_rate" : 0.05,
    #"alpha": 0.1
#}
#evals_result = {}
#num_boost_round=100000
#for i , (train_index, test_index) in enumerate(kf.split(X_train, y_train)):
#    X_train2= X_train.iloc[train_index,:]
#    y_train2= y_train.iloc[train_index]
#    X_test2= X_train.iloc[test_index,:]
#    y_test2= y_train.iloc[test_index]
#    xgb_train = xgb.DMatrix(X_train2, label = y_train2)
#    xgb_eval = xgb.DMatrix(X_test2, label = y_test2)
#    watchlist = [(xgb_train, "train"), (xgb_eval, "eval")]
    
#    clf = xgb.train(
#        xgb_params, xgb_train, num_boost_round, watchlist,
#        early_stopping_rounds=10,
        #evals_result=evals_result,
        #feval=crps_score,
#    )
    
#    models.append(clf)
#    temp_predict = clf.predict(xgb_eval, ntree_limit=clf.best_ntree_limit)
#    score = simple_crps(y_test2, temp_predict)
#    print(score)
#    crps_scores.append(score)
#    average_crps += score / n_folds
#    if i == 0:
#        feature_importance_df = pd.DataFrame(clf.get_score(importance_type="total_gain").items(), columns =["Features", "Fold_"+str(i+1)])
#    else:
#        feature_importance_df["Fold_"+str(i+1)] = list(clf.get_score(importance_type="total_gain").values())
    
#feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
#feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
#feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]
#print('crps list:', crps_scores)
#print('average crps:',average_crps)

In [9]:
# modified CV
df_20172018 = pd.concat([X_train, y_train], axis=1)
df_2017 = df_20172018[df_20172018.Season==2017].copy()
df_2018 = df_20172018[df_20172018.Season==2018].copy()
X_2017 = df_2017.drop(['Yards'],axis=1)
y_2017 = df_2017.Yards
X_2018 = df_2018.drop(['Yards'],axis=1)
y_2018 = df_2018.Yards
groups = np.array(X_2018.Week)
n_folds=3
gkf=GroupKFold(n_splits = n_folds)
average_crps = 0
models = []
crps_scores = []
xgb_params = {
    "objective" : "multi:softprob",
    "eval_metric" : "mlogloss", 
    "max_depth" : 4,
    "boosting": 'gbdt',
    "num_class": 199,
    "num_leaves" : 13,
    "learning_rate" : 0.05,
    #"alpha": 0.1
}
#evals_result = {}
num_boost_round=100000
# 2017 data: training only, 2018 data: separate by cross validation
for i , (train_index, test_index) in enumerate(gkf.split(X_2018, y_2018, groups)):
    X_train_2018 = X_2018.iloc[train_index,:]
    y_train_2018 = y_2018.iloc[train_index]
    X_test2 = X_2018.iloc[test_index,:]
    y_test2 = y_2018.iloc[test_index]
    X_train2 = pd.concat([X_2017, X_train_2018])
    y_train2 = pd.concat([y_2017, y_train_2018])
    xgb_train = xgb.DMatrix(X_train2, label = y_train2)
    xgb_eval = xgb.DMatrix(X_test2, label = y_test2)
    watchlist = [(xgb_train, "train"), (xgb_eval, "eval")]
    
    clf = xgb.train(
        xgb_params, xgb_train, num_boost_round, watchlist,
        early_stopping_rounds=10,
        #evals_result=evals_result,
        #feval=crps_score,
    )
    
    models.append(clf)
    temp_predict = clf.predict(xgb_eval, ntree_limit=clf.best_ntree_limit)
    score = simple_crps(y_test2, temp_predict)
    print(score)
    crps_scores.append(score)
    average_crps += score / n_folds
    if i == 0:
        feature_importance_df = pd.DataFrame(clf.get_score(importance_type="total_gain").items(), columns =["Features", "Fold_"+str(i+1)])
    else:
        feature_importance_df["Fold_"+str(i+1)] = list(clf.get_score(importance_type="total_gain").values())
    
feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]
print('crps list:', crps_scores)
print('average crps:',average_crps)

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	train-mlogloss:4.86322	eval-mlogloss:4.90261
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 10 rounds.
[1]	train-mlogloss:4.61548	eval-mlogloss:4.67846
[2]	train-mlogloss:4.43121	eval-mlogloss:4.51253
[3]	train-mlogloss:4.28263	eval-mlogloss:4.38009
[4]	train-mlogloss:4.1573	eval-mlogloss:4.27065
[5]	train-mlogloss:4.05005	eval-mlogloss:4.17469
[6]	train-mlogloss:3.9554	eval-mlogloss:4.09295
[7]	train-mlogloss:3.87099	eval-mlogloss:4.01967
[8]	train-mlogloss:3.79488	eval-mlogloss:3.9531
[9]	train-mlogloss:3.726	eval-mlogloss:3.89337
[10]	train-mlogloss:3.66263	eval-mlogloss:3.83896
[11]	train-mlogloss:3.60424	eval-mlogloss:3.78848
[12]	train-mlogloss:3.55003	eval-mlogloss:3.74284
[13]	train-mlogloss:3.49948	eval-mlogloss:3.69963
[14]	train-mlogloss:3.45216	eval-mlogloss:3.66007
[15]	train-mlogloss:3.40806	eval-mlogloss:3.62364
[16]	train-mlogloss:3.36632	eval-mlogloss:3.58903
[17]	train-mlo

- benchmark (Cv: 5 kfolds)
- xgb_params = { "max_depth" : 4,"boosting": 'gbdt',"num_class": 199,"num_leaves" : 13,"learning_rate" : 0.05}
- crps list: [0.012270968531269785, 0.012060295219752368, 0.011752867464111713, 0.014142862399415956, 0.013976765969843291]
- average crps: 0.012840751916878623
- 
- after modifications (Cv: 3 kfolds of 2018 data)
- xgb_params = { "max_depth" : 4,"boosting": 'gbdt',"num_class": 199,"num_leaves" : 13,"learning_rate" : 0.05}
- crps list: [0.013446466412538861, 0.014015271271516916, 0.013837981257613366]
- average crps: 0.013766572980556382
- 
- after modifications (Cv: 3 groupkfolds of 2018 data by weak)
- xgb_params = { "max_depth" : 4,"boosting": 'gbdt',"num_class": 199,"num_leaves" : 13,"learning_rate" : 0.05}
- crps list: [0.01380327268862825, 0.013758958618303138, 0.013717807739849194]
- average crps: 0.013760013015593528

In [10]:
X_train.columns

Index(['Team', 'S', 'A', 'Dis', 'NflId', 'JerseyNumber', 'Season', 'Down',
       'Distance', 'HomeScoreBeforePlay', 'VisitorScoreBeforePlay',
       'OffenseFormation', 'OffensePersonnel', 'DefendersInTheBox',
       'DefensePersonnel', 'PlayerWeight', 'PlayerCollegeName', 'Week',
       'Stadium', 'Location', 'StadiumType', 'GameWeather', 'Temperature',
       'Humidity', 'WindSpeed', 'WindDirection', 'YardLine', 'X', 'Y',
       'Orientation', 'Dir', 'age', 'Momentum', 'Dir_cos', 'Momentum_cos',
       'AttackAngle', 'time_end', 'Grass', 'DefendersInTheBox_vs_Distance',
       'DisToQB', 'runner_height', 'DefenseAveX', 'DefenseAveY',
       'DefenseAveMomentum', 'OffenseAveMomentum', 'DefenseAveMomentum_cos',
       'OffenseAveMomentum_cos', 'DefenseStdX', 'OffenseStdX', 'DefenseStdY',
       'OffenseStdY', 'DefenseStdMomentum', 'OffenseStdMomentum',
       'DefenseStdMomentum_cos', 'OffenseStdMomentum_cos',
       'RunnerToDefenseCentoid', 'RunnerToOffenseCentoid', 'DefenseSpreadX'

In [11]:
feature_importance_df.sort_values("Average").head(70).reset_index(drop=True)

,Features,Fold_1,Fold_2,Fold_3,Average,Std,Cv
0,Team,392.193866,276.701533,244.194768,304.363389,63.507569,0.208657
1,Season,324.355640,688.382362,246.100016,419.612672,192.715412,0.459270
2,OffenseFormation,759.100480,445.103892,276.926695,493.710356,199.824642,0.404741
3,OffensePersonnel,1225.864281,1536.192020,1146.176500,1302.744267,168.247686,0.129149
4,runner_height,1839.662330,1775.259249,812.260127,1475.727235,469.878276,0.318405
5,DefensePersonnel,1259.510762,1297.702347,2224.133422,1593.782177,445.998257,0.279836
6,Week,2219.123260,2401.852176,1709.622455,2110.199297,292.909336,0.138806
7,Grass,448.714780,1417.987051,5165.542024,2344.081285,2033.937513,0.867691
8,Down,626.183100,3343.087956,3252.070009,2407.113689,1259.856179,0.523389
9,PlayerCollegeName,4251.109638,1449.461917,1639.521594,2446.697716,1278.269001,0.522447


# prediction

In [12]:
for (test_df, sample_prediction_df) in env.iter_test():
    X_test = transform_data(test_df)
    for f in X_test.columns:
        if X_test[f].dtype=='object':
            X_test[f] = X_test[f].map(lambda x:x if x in set(X_train[f]) else -999)
    for f in X_test.columns:
        if X_test[f].dtype=='object': 
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(X_train[f])+[-999])
            X_test[f] = lbl.transform(list(X_test[f])) 
    pred_value = 0
    dtest = xgb.DMatrix(X_test)
    for model in models:
        pred_value += model.predict(dtest, ntree_limit = model.best_ntree_limit)[0] / n_folds
    pred_data = np.clip(np.cumsum(pred_value), 0, 1)
    pred_data = np.array(pred_data).reshape(1,199)
    pred_target = pd.DataFrame(index = sample_prediction_df.index, \
                               columns = sample_prediction_df.columns, \
                               data = pred_data)
    env.predict(pred_target)
env.write_submission_file()

Your submission file has been saved!  Once you `Commit` your Notebook and it finishes running, you can submit the file to the competition from the Notebook Viewer `Output` tab.
